# Предсказание активности молекул по отношению к таргету

*При подготовке ноутбука использовались данные из соревнования [Global AI Challenge](https://codenrock.com/contests/global-ai#/)* 

Целью данной задачи является предсказание активности молекулы лиганда по отношению к таргету - Covid 19

![](https://cloudfront.jove.com/files/media/science-education/science-education-thumbs/11513.jpg)

## План анализа данных:

  1. Загрузить данные для обучения
  2. Обработать данные перед обучением модели
  3. Обучить модель на обучающей выборке
  4. Загрузить и предобработать данные для тестирования
  5. Провалидировать модель на тестовой выборке


# 0. Установка и импорт библиотек

In [2]:
# Grab Jaime's excellent condacolab package: https://github.com/jaimergp/condacolab
# Note: you should probably read the README file at that repo.
# !pip3 install -q condacolab
# import condacolab
# condacolab.install()

You should consider upgrading via the '/home/ivankozlov98/environments/my_env/bin/python -m pip install --upgrade pip' command.


RuntimeError: This module must ONLY run as part of a Colab notebook!

In [3]:
# !mamba install -c conda-forge rdkit chembl_structure_pipeline

/bin/bash: mamba: command not found


In [4]:
!pip3 install  dgl
!pip3 install dgllife

     |████████████████████████████████| 4.5 MB 1.6 MB/s eta 0:00:01     |█████████████████████████████▋  | 4.1 MB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 554 kB/s eta 0:00:01
You should consider upgrading via the '/home/ivankozlov98/environments/my_env/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 138 kB 595 kB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 1.6 MB/s eta 0:00:01     |██                              | 102 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 199 kB 6.2 MB/s eta 0:00:01
  Created wheel for dgllife: filename=dgllife-0.2.9-py3-none-any.whl size=219060 sha256=bc09e021d374ff8a97c6371f8fe4ccc151578ed5aeeb8832f62d2b3e7bce6f92
  Stored in directory: /home/ivankozlov98/.cache/pip/wheels/46/54/a3/ef7d72bda8e3345f5568be4c827533bd1d7c9406222866750e
Successfully built dgllife
You should consider upgrading via the '/home/ivankozlov98/environments/my_env/bin/python -m pip in

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | ^C
failed

CondaError: KeyboardInterrupt



In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "end_to_end_project"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# 1. Загрузка данных

In [2]:
!wget https://www.dropbox.com/s/48c34raijlxc0nw/train.csv
!wget https://www.dropbox.com/s/297trreazro8ivr/test_labels.csv

--2022-03-22 18:58:00--  https://www.dropbox.com/s/48c34raijlxc0nw/train.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.71.18, 2620:100:6034:18::a27d:5412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.71.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/48c34raijlxc0nw/train.csv [following]
--2022-03-22 18:58:01--  https://www.dropbox.com/s/raw/48c34raijlxc0nw/train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucaa0f89f1345e19d437b7543b93.dl.dropboxusercontent.com/cd/0/inline/Bh8GkGzjjkCx62AnMbj7K5gS4vM0i1lFlzbnYrjNuNw2oudarLKs0jdil-Ue9wjwbV5UGkxa7VyTYE6eKNeU2lG7K4aZo5CJp8tDsCXoQg0pk-yBNbTFtMd2DfyXyHEVxt7zXGh9DBgGNx6t1e5dnAeHL6Iwbb0WY6iTOav7XUv1lw/file# [following]
--2022-03-22 18:58:02--  https://ucaa0f89f1345e19d437b7543b93.dl.dropboxusercontent.com/cd/0/inline/Bh8GkGzjjkCx62AnMbj7K5gS4vM0i1lFlzbnYrjNuNw2oudarLKs0jdil-Ue9wjwbV5UGkxa7V

In [3]:
DATA_PATH = "./"
TRAIN_FILE = "train.csv"
TEST_FILE = "test_labels.csv"

SMILES_COLUMN = "smiles"
TARGET_COLUMN = "Active"

In [4]:
import pandas as pd

def load_train_test_data():
    train_csv_path = os.path.join(DATA_PATH, TRAIN_FILE)
    test_csv_path = os.path.join(DATA_PATH, TEST_FILE)
    train_data = pd.read_csv(train_csv_path, index_col = 0)
    test_data = pd.read_csv(test_csv_path,index_col = 0)
    return train_data.rename(columns = {"Smiles":SMILES_COLUMN}), test_data.rename(columns = {"Smiles":SMILES_COLUMN})

## 1.1 Анализ данных, формулировка задачи машинного обучения

Молекулы представлены в виде [SMILES нотации](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system).

![](https://upload.wikimedia.org/wikipedia/commons/thumb/0/00/SMILES.png/450px-SMILES.png)

In [5]:
train_data, test_data = load_train_test_data()
train_data.head()

,smiles,Active
0,COc1ccc2[nH]cc(CCN)c2c1,False
1,CCCN1CCC[C@H](c2cccc(O)c2)C1.Cl,False
2,O=C(NO)c1cnc(N2CCN(S(=O)(=O)c3ccc4ccccc4c3)CC2...,False
3,Nc1cccc(CNC(=O)c2ccc(Oc3ccc(OCc4cccc(F)c4)cc3)...,False
4,Fc1ccccc1CNCc1ccc(-c2ccnc3[nH]ccc23)cc1,False


In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5557 entries, 0 to 5556
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   smiles  5557 non-null   object
 1   Active  5557 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 92.3+ KB


In [7]:
train_data[TARGET_COLUMN].value_counts()

False    5351
True      206
Name: Active, dtype: int64

In [9]:
!pip3 install rdkit-pypi

  Using cached rdkit_pypi-2021.9.5.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (22.5 MB)
You should consider upgrading via the '/home/ivankozlov98/environments/my_env/bin/python -m pip install --upgrade pip' command.


## 1.2 Предобработка данных

In [10]:
from rdkit import Chem
from rdkit.Chem.SaltRemover import SaltRemover

In [11]:
def remove_salts_and_canonicalized(smiles: str):
  remover = SaltRemover(defnData="[Cl,Br]")
  mol = Chem.MolFromSmiles(smiles)
  res = remover.StripMol(mol)
  processed_smiles = Chem.MolToSmiles(mol)
  return processed_smiles

In [12]:
train_data[SMILES_COLUMN] = list(map(remove_salts_and_canonicalized, train_data[SMILES_COLUMN]))
test_data[SMILES_COLUMN] = list(map(remove_salts_and_canonicalized, test_data[SMILES_COLUMN]))

In [13]:
def change_str_target_to_int(targets: pd.Series):
  target_map = {True: 1, False: 0}
  processed_targets = targets.map(target_map)
  return processed_targets.values

In [14]:
train_data[TARGET_COLUMN] = change_str_target_to_int(train_data[TARGET_COLUMN])
test_data[TARGET_COLUMN] = change_str_target_to_int(test_data[TARGET_COLUMN])

In [15]:
#TODO CHECK SaltRemover
train_data.head()

,smiles,Active
0,COc1ccc2[nH]cc(CCN)c2c1,0
1,CCCN1CCC[C@H](c2cccc(O)c2)C1.Cl,0
2,O=C(NO)c1cnc(N2CCN(S(=O)(=O)c3ccc4ccccc4c3)CC2...,0
3,Nc1cccc(CNC(=O)c2ccc(Oc3ccc(OCc4cccc(F)c4)cc3)...,0
4,Fc1ccccc1CNCc1ccc(-c2ccnc3[nH]ccc23)cc1,0


## 1.3 Feature engineering

Молекулу можно представить в виде фингерпринта - вектора свойств, полученного по определенному алгоритму.

Мы будем считать фингерпринты при помощи библиотеки RDKit. Про различные фингерпринты и их описание можно почитать тут - https://www.rdkit.org/docs/GettingStartedInPython.html#fingerprinting-and-molecular-similarity

![](https://sun9-64.userapi.com/impf/_8Zy5WO6Mt0SIPx1YS02DeErAoZ0RHcwgc-kZg/Md98bNVzBg0.jpg?size=831x415&quality=96&sign=cb20481128a04ff523fd662dd0e604ab&type=album)


### Моргановские фингерпринты (ECFP)

![](https://d3i71xaburhd42.cloudfront.net/52adf3589e8b7b9855353e5815669258ef6e3405/6-Figure2-1.png)

In [16]:
from enum import Enum
from functools import partial
from rdkit import Chem, DataStructs
from rdkit.DataStructs import ExplicitBitVect
from rdkit.Chem import AllChem, MACCSkeys
from typing import List


In [17]:
class FingerprintsNames(Enum):
  ECFP4 = "morgan_2_2048"
  RDKitFP = "RDKFingerprint"
  TOPOTORSION = "topological_torsion"
  MACCS = "MACCSkeys"
  PATTERN = "PatternFingerprint"
  ATOMPAIR = "AtomPairFingerprint"



FINGERPRINTS_METHODS = {
    FingerprintsNames.ECFP4: partial(AllChem.GetMorganFingerprintAsBitVect, radius=2, nBits=2048),
    FingerprintsNames.RDKitFP: Chem.RDKFingerprint,#TODO
    FingerprintsNames.TOPOTORSION: AllChem.GetHashedTopologicalTorsionFingerprintAsBitVect,#TODO
    FingerprintsNames.MACCS: MACCSkeys.GenMACCSKeys,#TODO
    FingerprintsNames.PATTERN: Chem.PatternFingerprint,#TODO
    FingerprintsNames.ATOMPAIR: AllChem.GetHashedAtomPairFingerprintAsBitVect}#TODO


In [18]:
fingerprint_type_name = FingerprintsNames.ECFP4
fingerprint_type_method = FINGERPRINTS_METHODS[fingerprint_type_name]

In [19]:
def bit_vectors_to_numpy_arrays(fps: List[ExplicitBitVect]) -> np.array:
    output_arrays = [np.zeros((1,)) for i in range(len(fps))]
    _ = list(
        map(lambda fp_output_array: DataStructs.ConvertToNumpyArray(fp_output_array[0], fp_output_array[1]),
            zip(fps, output_arrays)))
    return np.asarray(output_arrays)

def get_np_array_of_fps(fp_type, smiles: List[str]):
    # Calculate the morgan fingerprint
    mols = [Chem.MolFromSmiles(m) for m in smiles]
    fp = list(map(fp_type, mols))
    return bit_vectors_to_numpy_arrays(fp)

In [20]:
train_fp = get_np_array_of_fps(fp_type=fingerprint_type_method, smiles=train_data[SMILES_COLUMN])
test_fp = get_np_array_of_fps(fp_type=fingerprint_type_method, smiles=test_data[SMILES_COLUMN])

In [21]:
y_train = train_data[TARGET_COLUMN]
y_test = test_data[TARGET_COLUMN]

# 2. Подготовка к обучению модели

## 2.1 Кросс-валидация

![](https://pubs.rsc.org/image/article/2018/SC/c7sc02664a/c7sc02664a-f3_hi-res.gif)

In [26]:
!pip3 install torch

     |████████████████████████████████| 750.6 MB 6.0 kB/s eta 0:00:017   |█                               | 25.9 MB 236 kB/s eta 0:51:00     |█▋                              | 36.6 MB 2.1 MB/s eta 0:05:42     |██▉                             | 65.3 MB 512 kB/s eta 0:22:17     |███                             | 70.6 MB 856 kB/s eta 0:13:14     |███▏                            | 75.5 MB 1.3 MB/s eta 0:08:32     |██████▋                         | 155.6 MB 2.3 MB/s eta 0:04:17     |███████▋                        | 178.6 MB 2.7 MB/s eta 0:03:29     |████████                        | 185.6 MB 3.2 MB/s eta 0:02:59     |██████████▊                     | 251.7 MB 1.7 MB/s eta 0:04:57     |███████████▍                    | 267.5 MB 2.1 MB/s eta 0:03:50     |████████████▍                   | 289.5 MB 1.1 MB/s eta 0:06:54     |████████████▌                   | 293.9 MB 7.3 MB/s eta 0:01:03     |████████████▋                   | 294.6 MB 7.3 MB/s eta 0:01:03     |██████████████▌                 | 

In [28]:
from dgllife.utils import ScaffoldSplitter

In [29]:
class ScaffoldCVSklearn:
    def __init__(self, data, k_folds):
        self.scaffold_splits = ScaffoldSplitter.k_fold_split(data, k=k_folds)

    def split(self):
        indices_splits = []
        for train_data, val_data in self.scaffold_splits:
          train_indices = train_data.indices
          val_indices = val_data.indices
          indices_splits.append((train_indices, val_indices))
        return indices_splits

    def convert_data_to_indices(self, dataset):
        indices = [index for index, row in dataset.iterrows()]
        return indices


In [65]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold

In [66]:
# cv_model = ScaffoldCVSklearn(train_data, k_folds=3) 
# cv = cv_model.split()
cv_model = KFold(n_splits=3) 
cv = cv_model.split(train_data)

## 2.2 Установка модели

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, nthread=1, use_label_encoder=False)

In [ ]:
params = {
        'max_depth': [10],
        'n_estimators': [100]
    }

## 2.3 Поиск параметров

In [40]:
from sklearn.model_selection import GridSearchCV

In [23]:
grid_search = GridSearchCV(xgb, param_grid=params, scoring='accuracy', n_jobs=4,
                               cv=cv, verbose=10000)

NameError: name 'GridSearchCV' is not defined

# Подбор параметров модели

In [30]:
print('\n Start Grid Search')
grid_search.fit(train_fp, y_train)


 Start Grid Search
Fitting 3 folds for each of 1 candidates, totalling 3 fits


GridSearchCV(cv=[([5, 6, 13, 81, 96, 126, 149, 160, 163, 175, 194, 196, 230,
                   248, 260, 280, 283, 287, 301, 320, 329, 353, 392, 393, 394,
                   409, 430, 443, 462, 490, ...],
                  [22, 49, 52, 67, 103, 107, 154, 184, 188, 202, 214, 225, 226,
                   229, 236, 254, 266, 292, 293, 302, 308, 313, 331, 382, 383,
                   402, 454, 474, 477, 482, ...]),
                 ([22, 49, 52, 67, 103, 107, 154, 184, 188, 202, 214, 225, 226,
                   229, 236, 254, 266, 292, 293, 302, 308, 313, 331,...
                   402, 454, 474, 477, 482, ...],
                  [170, 537, 714, 962, 1160, 1556, 1726, 1798, 1995, 2549, 2551,
                   2639, 2704, 2819, 2933, 3128, 3141, 3156, 3242, 3364, 3397,
                   3435, 3474, 3683, 3846, 4470, 4552, 4649, 4674, 4835, ...])],
             estimator=XGBClassifier(learning_rate=0.02, n_estimators=600,
                                     nthread=1, use_label_encoder=

In [31]:

print('\n All results:')
print(grid_search.cv_results_)
print('\n Best estimator:')
print(grid_search.best_estimator_)
print('\n Best normalized score')
print(grid_search.best_score_)
print('\n Best hyperparameters:')
print(grid_search.best_params_)


 All results:
{'mean_fit_time': array([107.78480299]), 'std_fit_time': array([0.16640527]), 'mean_score_time': array([0.09587534]), 'std_score_time': array([0.03414906]), 'param_max_depth': masked_array(data=[10],
             mask=[False],
       fill_value='?',
            dtype=object), 'param_n_estimators': masked_array(data=[100],
             mask=[False],
       fill_value='?',
            dtype=object), 'params': [{'max_depth': 10, 'n_estimators': 100}], 'split0_test_score': array([0.95412844]), 'split1_test_score': array([0.96706263]), 'split2_test_score': array([0.96976242]), 'mean_test_score': array([0.96365116]), 'std_test_score': array([0.00682319]), 'rank_test_score': array([1], dtype=int32)}

 Best estimator:
XGBClassifier(learning_rate=0.02, max_depth=10, nthread=1,
              use_label_encoder=False)

 Best normalized score
0.9636511647875509

 Best hyperparameters:
{'max_depth': 10, 'n_estimators': 100}


# Обучение и оценка модели

In [32]:
xgb = XGBClassifier(max_depth=10, n_estimators=100, learning_rate=0.02,  nthread=1, use_label_encoder=False)

In [33]:
xgb.fit(train_fp, y_train)

XGBClassifier(learning_rate=0.02, max_depth=10, nthread=1,
              use_label_encoder=False)

In [34]:
test_predictions = xgb.predict(test_fp)

In [79]:
from sklearn.metrics import f1_score

In [36]:
score = f1_score(y_test, test_predictions)
print(f"Best model test f1 score is {round(score, 3)}")

Best model test f1 score is 0.265


# Задание (10 баллов + 3 бонусных)
1. (3 балла) Добавить решение проблемы несбалансированной классификации

Варианты:
* UnderSampling
* OverSampling
* SMOTE
* Внутренние инструменты модели (`scale_pos_weight`)

2. (2 балла) Использовать еще 2 вида фингерпринтов из `FingerprintsNames`

3. (3 балла) Получить f1-score на тестовом датасете больше 0.35

Варианты:
* Увеличить количество параметров в подборе гиперпараметров
* Использовать другие алгоритмы подбора гиперпараметров (например, [RandomizedSearch](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html))
* Использовать другие модели (Random Forest, SVC, MLPClassifier, etc)

4. (2 балла) Логирование

В качестве финального результата предоставьте таблицу (можно `pd.DataFrame`) c колонками: Model, Fingerprint, Best Parameters, Mean Cross-Validation Score, Std Cross-Validation Score, Test Score 

Проанализируйте результаты: 
* Какие фингерпринты дали лучший результат?
* Какая модель дала лучший результат.
* Коррелируют ли скоры на кросс-валидации и тестовой выборке?

5. (Бонус +3 балла) Получить f1-score на тестовом датасете больше 0.45

In [35]:
from catboost import CatBoostClassifier, Pool

In [179]:
# False    5351
# True      206
zero = 5351
one = 206
scale_pos_weight = (one + zero) / one 

In [216]:
def task5(fingerprint_method, name_fp, iterations, verbose):
    train_fp = get_np_array_of_fps(fp_type=fingerprint_method, smiles=train_data[SMILES_COLUMN])
    test_fp = get_np_array_of_fps(fp_type=fingerprint_method, smiles=test_data[SMILES_COLUMN])
    model = CatBoostClassifier(depth=7, iterations=iterations, verbose=verbose, learning_rate=0.3, scale_pos_weight=scale_pos_weight, random_state=53)
    model.fit(train_fp, y_train)
    test_predictions = model.predict(test_fp)
    score = f1_score(y_test, test_predictions)
    print(f"For {name_fp}: ")
    print(f"Best model on maccs test f1 score is {round(score, 3)}")

In [213]:
fingerprint_name_maccs = FingerprintsNames.MACCS
fingerprint_name_ATOMPAIR = FingerprintsNames.ATOMPAIR

fingerprint_method_maccs = FINGERPRINTS_METHODS[fingerprint_name_maccs]
fingerprint_method_ATOMPAIR = FINGERPRINTS_METHODS[fingerprint_name_ATOMPAIR]

In [214]:
task5(fingerprint_method_maccs, "Maccs", iterations=10000, verbose=False)

For Maccs: 
Best model on maccs test f1 score is 0.364


In [217]:
task5(fingerprint_method_ATOMPAIR, "ATOMPAIR", iterations=1000, verbose=True)

0:	learn: 0.6293502	total: 167ms	remaining: 2m 46s
1:	learn: 0.5887278	total: 327ms	remaining: 2m 43s
2:	learn: 0.5679900	total: 519ms	remaining: 2m 52s
3:	learn: 0.5331439	total: 683ms	remaining: 2m 50s
4:	learn: 0.5040710	total: 856ms	remaining: 2m 50s
5:	learn: 0.4737354	total: 1.02s	remaining: 2m 49s
6:	learn: 0.4476091	total: 1.2s	remaining: 2m 50s
7:	learn: 0.4307038	total: 1.36s	remaining: 2m 48s
8:	learn: 0.4088548	total: 1.53s	remaining: 2m 48s
9:	learn: 0.3925072	total: 1.69s	remaining: 2m 47s
10:	learn: 0.3870513	total: 1.86s	remaining: 2m 47s
11:	learn: 0.3774997	total: 2.02s	remaining: 2m 46s
12:	learn: 0.3549959	total: 2.23s	remaining: 2m 49s
13:	learn: 0.3438299	total: 2.4s	remaining: 2m 49s
14:	learn: 0.3355555	total: 2.58s	remaining: 2m 49s
15:	learn: 0.3211676	total: 2.74s	remaining: 2m 48s
16:	learn: 0.3155368	total: 2.92s	remaining: 2m 48s
17:	learn: 0.2961736	total: 3.08s	remaining: 2m 47s
18:	learn: 0.2903274	total: 3.26s	remaining: 2m 48s
19:	learn: 0.2805088	tot

159:	learn: 0.0071600	total: 28.4s	remaining: 2m 29s
160:	learn: 0.0071599	total: 28.6s	remaining: 2m 28s
161:	learn: 0.0071282	total: 28.7s	remaining: 2m 28s
162:	learn: 0.0070760	total: 28.9s	remaining: 2m 28s
163:	learn: 0.0069830	total: 29.1s	remaining: 2m 28s
164:	learn: 0.0069437	total: 29.3s	remaining: 2m 28s
165:	learn: 0.0069437	total: 29.4s	remaining: 2m 27s
166:	learn: 0.0068643	total: 29.6s	remaining: 2m 27s
167:	learn: 0.0068642	total: 29.8s	remaining: 2m 27s
168:	learn: 0.0068642	total: 29.9s	remaining: 2m 27s
169:	learn: 0.0068641	total: 30.1s	remaining: 2m 27s
170:	learn: 0.0068641	total: 30.3s	remaining: 2m 26s
171:	learn: 0.0068150	total: 30.5s	remaining: 2m 26s
172:	learn: 0.0068150	total: 30.6s	remaining: 2m 26s
173:	learn: 0.0068150	total: 30.8s	remaining: 2m 26s
174:	learn: 0.0068150	total: 31s	remaining: 2m 26s
175:	learn: 0.0068150	total: 31.2s	remaining: 2m 26s
176:	learn: 0.0068148	total: 31.4s	remaining: 2m 25s
177:	learn: 0.0068148	total: 31.6s	remaining: 2m

317:	learn: 0.0063513	total: 56.3s	remaining: 2m
318:	learn: 0.0063513	total: 56.5s	remaining: 2m
319:	learn: 0.0063513	total: 56.7s	remaining: 2m
320:	learn: 0.0063511	total: 56.9s	remaining: 2m
321:	learn: 0.0063511	total: 57.1s	remaining: 2m
322:	learn: 0.0063511	total: 57.2s	remaining: 1m 59s
323:	learn: 0.0063510	total: 57.4s	remaining: 1m 59s
324:	learn: 0.0063510	total: 57.6s	remaining: 1m 59s
325:	learn: 0.0063510	total: 57.8s	remaining: 1m 59s
326:	learn: 0.0063510	total: 57.9s	remaining: 1m 59s
327:	learn: 0.0063510	total: 58.1s	remaining: 1m 59s
328:	learn: 0.0063510	total: 58.3s	remaining: 1m 58s
329:	learn: 0.0063510	total: 58.5s	remaining: 1m 58s
330:	learn: 0.0063111	total: 58.7s	remaining: 1m 58s
331:	learn: 0.0063111	total: 58.9s	remaining: 1m 58s
332:	learn: 0.0063111	total: 59s	remaining: 1m 58s
333:	learn: 0.0063110	total: 59.3s	remaining: 1m 58s
334:	learn: 0.0063110	total: 59.5s	remaining: 1m 58s
335:	learn: 0.0063109	total: 59.7s	remaining: 1m 57s
336:	learn: 0.0

472:	learn: 0.0062232	total: 1m 24s	remaining: 1m 33s
473:	learn: 0.0062231	total: 1m 24s	remaining: 1m 33s
474:	learn: 0.0062231	total: 1m 24s	remaining: 1m 33s
475:	learn: 0.0062231	total: 1m 24s	remaining: 1m 33s
476:	learn: 0.0062231	total: 1m 25s	remaining: 1m 33s
477:	learn: 0.0062231	total: 1m 25s	remaining: 1m 33s
478:	learn: 0.0062231	total: 1m 25s	remaining: 1m 33s
479:	learn: 0.0062231	total: 1m 25s	remaining: 1m 32s
480:	learn: 0.0062231	total: 1m 25s	remaining: 1m 32s
481:	learn: 0.0062231	total: 1m 26s	remaining: 1m 32s
482:	learn: 0.0062231	total: 1m 26s	remaining: 1m 32s
483:	learn: 0.0062231	total: 1m 26s	remaining: 1m 32s
484:	learn: 0.0062231	total: 1m 26s	remaining: 1m 31s
485:	learn: 0.0062230	total: 1m 26s	remaining: 1m 31s
486:	learn: 0.0062230	total: 1m 26s	remaining: 1m 31s
487:	learn: 0.0062230	total: 1m 27s	remaining: 1m 31s
488:	learn: 0.0062230	total: 1m 27s	remaining: 1m 31s
489:	learn: 0.0062230	total: 1m 27s	remaining: 1m 31s
490:	learn: 0.0062230	total:

625:	learn: 0.0061284	total: 1m 51s	remaining: 1m 6s
626:	learn: 0.0061284	total: 1m 51s	remaining: 1m 6s
627:	learn: 0.0061284	total: 1m 51s	remaining: 1m 6s
628:	learn: 0.0061284	total: 1m 51s	remaining: 1m 5s
629:	learn: 0.0061283	total: 1m 51s	remaining: 1m 5s
630:	learn: 0.0061283	total: 1m 52s	remaining: 1m 5s
631:	learn: 0.0061283	total: 1m 52s	remaining: 1m 5s
632:	learn: 0.0061283	total: 1m 52s	remaining: 1m 5s
633:	learn: 0.0061283	total: 1m 52s	remaining: 1m 4s
634:	learn: 0.0061282	total: 1m 52s	remaining: 1m 4s
635:	learn: 0.0061282	total: 1m 52s	remaining: 1m 4s
636:	learn: 0.0061281	total: 1m 53s	remaining: 1m 4s
637:	learn: 0.0061281	total: 1m 53s	remaining: 1m 4s
638:	learn: 0.0061281	total: 1m 53s	remaining: 1m 4s
639:	learn: 0.0061281	total: 1m 53s	remaining: 1m 3s
640:	learn: 0.0061010	total: 1m 53s	remaining: 1m 3s
641:	learn: 0.0061010	total: 1m 53s	remaining: 1m 3s
642:	learn: 0.0061010	total: 1m 54s	remaining: 1m 3s
643:	learn: 0.0061010	total: 1m 54s	remaining:

783:	learn: 0.0059850	total: 2m 18s	remaining: 38.3s
784:	learn: 0.0059850	total: 2m 19s	remaining: 38.1s
785:	learn: 0.0059850	total: 2m 19s	remaining: 37.9s
786:	learn: 0.0059849	total: 2m 19s	remaining: 37.7s
787:	learn: 0.0059849	total: 2m 19s	remaining: 37.5s
788:	learn: 0.0059849	total: 2m 19s	remaining: 37.4s
789:	learn: 0.0059849	total: 2m 19s	remaining: 37.2s
790:	learn: 0.0059848	total: 2m 20s	remaining: 37s
791:	learn: 0.0059848	total: 2m 20s	remaining: 36.8s
792:	learn: 0.0059847	total: 2m 20s	remaining: 36.7s
793:	learn: 0.0059846	total: 2m 20s	remaining: 36.5s
794:	learn: 0.0059846	total: 2m 20s	remaining: 36.3s
795:	learn: 0.0059846	total: 2m 20s	remaining: 36.1s
796:	learn: 0.0059846	total: 2m 21s	remaining: 35.9s
797:	learn: 0.0059846	total: 2m 21s	remaining: 35.8s
798:	learn: 0.0059846	total: 2m 21s	remaining: 35.6s
799:	learn: 0.0059844	total: 2m 21s	remaining: 35.4s
800:	learn: 0.0059844	total: 2m 21s	remaining: 35.2s
801:	learn: 0.0059843	total: 2m 21s	remaining: 3

939:	learn: 0.0059411	total: 2m 45s	remaining: 10.6s
940:	learn: 0.0059411	total: 2m 46s	remaining: 10.4s
941:	learn: 0.0059411	total: 2m 46s	remaining: 10.2s
942:	learn: 0.0059411	total: 2m 46s	remaining: 10.1s
943:	learn: 0.0059411	total: 2m 46s	remaining: 9.89s
944:	learn: 0.0059410	total: 2m 46s	remaining: 9.71s
945:	learn: 0.0059410	total: 2m 47s	remaining: 9.53s
946:	learn: 0.0059410	total: 2m 47s	remaining: 9.36s
947:	learn: 0.0059410	total: 2m 47s	remaining: 9.18s
948:	learn: 0.0059409	total: 2m 47s	remaining: 9s
949:	learn: 0.0059409	total: 2m 47s	remaining: 8.83s
950:	learn: 0.0059408	total: 2m 47s	remaining: 8.65s
951:	learn: 0.0059408	total: 2m 48s	remaining: 8.48s
952:	learn: 0.0059406	total: 2m 48s	remaining: 8.3s
953:	learn: 0.0059406	total: 2m 48s	remaining: 8.13s
954:	learn: 0.0059406	total: 2m 48s	remaining: 7.96s
955:	learn: 0.0059405	total: 2m 49s	remaining: 7.78s
956:	learn: 0.0059405	total: 2m 49s	remaining: 7.61s
957:	learn: 0.0059405	total: 2m 49s	remaining: 7.4

In [ ]:
# TODO
# Я доделаю эту работу